## Imports

In [0]:
import sys
import os
sys.path.append(os.path.abspath('../utils/'))

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType
import json
from dwh_utils import *
from transformation_utils import *

## Assumption
- We have enabled CDC on the bronze table
- So in production, i would use readstream and handle CDC records through foreachbatch

In [0]:
df = spark.read.table("acme_ecommerce.bronze.customers")


## Transformations

In [0]:
df = deduplicate_data_by_time(spark=spark, df=df, partition_cols=["customer_id"], order_col="ingest_timestamp")
df = transform_names_column(spark=spark, df=df, name_column="customer_name")
df = drop_column(spark=spark, df=df, column="email")
df = drop_column(spark=spark, df=df, column="phone")
df = drop_column(spark=spark, df=df, column="address")

## Writing to delta table
- On the first run, we are inserting the data and post that it will always be upsert to ensure data quality

In [0]:
write_to_delta_with_cdc_by_name(
    spark=spark, df=df, catalog="acme_ecommerce", schema="silver", table="customers", merge_keys=["customer_id"], append_only=False
)